In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [60]:
data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_ids= test_data["PassengerId"]
#train_data.head()

def clean (data):
    data=data.drop(["PassengerId","Name","Ticket","Cabin"], axis=1)
    cols = ["SibSp","Parch","Fare","Age"]
    for col in cols:
        data[col].fillna(data[col].median(),inplace=True)
        data.Embarked.fillna("U",inplace=True)
    return data
        
test_data=clean(test_data)
train_data=clean(data)



In [53]:
test_data.head(7)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2
5,3,1,14.0,0,0,9.2250,2
6,3,0,30.0,0,0,7.6292,1


In [54]:
train_data.head(7)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
5,0,3,1,28.0,0,0,8.4583,1
6,0,1,1,54.0,0,0,51.8625,2


In [71]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]
for col in cols:
    test_data[col] = le.fit_transform(test_data[col])
    train_data[col] = le.fit_transform(train_data[col])
    #test_data[col]=le.transform(test_data[col])
    print (le.classes_)
    
#train_data.head(7)
test_data.head(7)

['female' 'male']
['C' 'Q' 'S' 'U']


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2
5,3,1,14.0,0,0,9.2250,2
6,3,0,30.0,0,0,7.6292,1


In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train_data["Survived"]
x = train_data.drop("Survived",axis=1)

X_train, X_val,Y_train,Y_val = train_test_split(X,y,test_size=0.2, random_state=42)

In [73]:
clf = LogisticRegression(random_state=0, max_iter=1000000).fit(X_train,Y_train)

In [74]:
predictions =clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(Y_val,predictions)


0.7932960893854749

In [75]:
test_data = test_data[['Pclass', 'Sex', 'SibSp', 'Parch']]


In [82]:
submission_preds=clf.predict(test_data)

In [84]:
df = pd.DataFrame({"PassengerId":test_ids.values,
                 "Survived":submission_preds,
                  })

In [86]:
df.to_csv('submission.csv', index=False)